In [1]:
from ipywidgets import widgets, HBox, Layout
import random
from traitlets import traitlets
from IPython.display import clear_output


def find_winner(b, computer, user):
    """Function that determines the winner of the RPS-game."""
    
    # Message that should be printed out
    tie_msg = "You've chosen {}. I've chosen {}. It's a tie!".format(user, computer)
    comp_win_msg = "You've chosen {}. I've chosen {}. I win this round!".format(user, computer)
    user_win_msg = "You've chosen {}. I've chosen {}. You win this round!".format(user, computer)
    
    # Dictionary that specifies who wins against who
    win_dict = {'Paper': ['Rock', 'Spock'],
                'Rock': ['Lizard', 'Scissors'],
                'Scissors': ['Paper', 'Lizard'],
                'Lizard': ['Spock', 'Paper'],
                'Spock': ['Scissors', 'Rock']}
    
    # Logic to determine the winner
    if computer == user:
        display(widgets.Label(tie_msg))
        b.score_update_comp = 0
        b.score_update_user = 0
    elif user in win_dict[computer]:
        display(widgets.Label(comp_win_msg))
        b.score_update_comp = 1
        b.score_update_user = 0
    else:
        display(widgets.Label(user_win_msg))
        b.score_update_comp = 0
        b.score_update_user = 1

    return b.score_update_comp, b.score_update_user


class LoadedButton(widgets.Button):
    """A button that can hold the result of the last game."""

    def __init__(self, score_update_comp = None, score_update_user = None, *args, **kwargs):
        super(LoadedButton, self).__init__(*args, **kwargs)
        
        self.add_traits(score_update_comp = traitlets.Any(score_update_comp))
        self.add_traits(score_update_user = traitlets.Any(score_update_user))

        
# Define the game class
class Game:
    # Initialize computer score and user score
    computer_score = 0
    user_score = 0
        
    def __init__(self, rounds_to_win = 2):
        # Optional parameter that can be used to alter the number of round to be played
        self.rounds_to_win = rounds_to_win
    
    # Method that defines what should if the user clicks on one of the buttons
    def on_button_clicked(self, b):
        """Function that defines what should happen if the user clicks on one of the buttons."""

        clear_output()
        
        user_choice = b.description
        computer_choice = random.choice(["Paper", "Rock", "Scissors", "Lizard", "Spock"])

        b.score_update_comp, b.score_update_user = find_winner(b, computer_choice, user_choice)

        self.computer_score = self.computer_score + b.score_update_comp
        self.user_score = self.user_score + b.score_update_user

        # Define button to restart the game
        restart_button = widgets.Button(description = "Try again.", button_style = 'success', icon = 'check')
        restart_button.on_click(self.another_set)

        # Determine what should happen when someone wins the best of three
        if self.user_score == self.rounds_to_win:
            win_msg = widgets.Label("So, you've won the best of {}! Congratulations!".format((self.rounds_to_win * 2) - 1))
            display(win_msg, restart_button)
        elif self.computer_score == self.rounds_to_win:
            lose_msg = widgets.Label("So, I've won the best of {}!".format((self.rounds_to_win * 2) - 1))
            display(lose_msg, restart_button)
        else:
            self.play()

  
    def play(self, b = None, first_round = False):
        """Method that defines the user interface and starts the game"""
        
        # Define the buttons for the game
        button_list = ['Paper', 'Rock', 'Scissors', 'Lizard', 'Spock']
        paper_button, rock_button, scissors_button, lizard_button, spock_button = [LoadedButton(description = x,
                                                                                                score_update_comp = 0,
                                                                                                score_update_user = 0,
                                                                                                button_style = 'info',
                                                                                                layout = Layout(width = '100px'))
                                                                                   for x in button_list]
        # Add functionality to the buttons
        [x.on_click(self.on_button_clicked) for x in [paper_button, rock_button, scissors_button, lizard_button, spock_button]]

        # Define the boxes that are used for the layout
        score_box = HBox([widgets.Button(description = "User score: {}".format(self.user_score),
                                         layout = Layout(width = '256px')),
                          widgets.Button(description = "Computer score: {}".format(self.computer_score),
                                         layout = Layout(width = '256px'))])
        
        button_box = HBox([rock_button, paper_button, scissors_button, lizard_button, spock_button])

        # Display the user interface
        if first_round:
            display(widgets.Label('Welcome to a game of Rock-Paper-Scissors-Lizard-Spock! Please choose your weapon.'))
            
        display(button_box, score_box)
    
    # Method that is called when the user wants to play another set (game)
    def another_set(self, b):
        self.computer_score = 0
        self.user_score = 0
        
        clear_output()
        self.play(first_round = True)
        
        
# Create an instance of the game and start playing :) - optionally you can pass the number of rounds you want to play to Game()
game = Game(rounds_to_win = 2)
game.play(first_round = True)

Label(value='Welcome to a game of Rock-Paper-Scissors-Lizard-Spock! Please choose your weapon.')